In [7]:
import pandas as pd
import traceback
import os,sys

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from MetaCreate import MetaCreate
from AudioManipulator import AudioManipulator
from preprocess_uitls import pad_audio_files, featurize
from utils import char_map,index_map, int_sequence_to_text ,text_to_int_sequence
from ctc_utils import ctc_lambda_func, add_ctc_loss, cnn_output_length


import librosa
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers  import (BatchNormalization, Conv1D, Dense, Input, 
    TimeDistributed, Activation, Bidirectional, SimpleRNN, GRU, LSTM)
    
# from keras.utils.vis_utils import plot_model


import _pickle as pickle
from numpy.lib.stride_tricks import as_strided

from keras.layers import (Input, Lambda)
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint   

In [64]:
def simple_rnn(input_dim, units, activation, output_dim=29):
    """ Build a recurrent network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # Add recurrent layer
    simp_rnn = GRU(units, activation=activation,
        return_sequences=True, implementation=2, name='rnn')(input_data)
    bn_rnn = BatchNormalization()(simp_rnn)
    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    # plot_model(model, to_file='models/model_1.png')
    return model

In [8]:
def model_2(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=29, dropout_rate=0.5, number_of_layers=2, 
    cell=GRU, activation='tanh'):
    """ Build a deep network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # TODO: Specify the layers in your network
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='layer_1_conv',
                     dilation_rate=1)(input_data)
    conv_bn = BatchNormalization(name='conv_batch_norm')(conv_1d)


    if number_of_layers == 1:
        layer = cell(units, activation=activation,
            return_sequences=True, implementation=2, name='rnn_1', dropout=dropout_rate)(conv_bn)
        layer = BatchNormalization(name='bt_rnn_1')(layer)
    else:
        layer = cell(units, activation=activation,
                    return_sequences=True, implementation=2, name='rnn_1', dropout=dropout_rate)(conv_bn)
        layer = BatchNormalization(name='bt_rnn_1')(layer)

        for i in range(number_of_layers - 2):
            layer = cell(units, activation=activation,
                        return_sequences=True, implementation=2, name='rnn_{}'.format(i+2), dropout=dropout_rate)(layer)
            layer = BatchNormalization(name='bt_rnn_{}'.format(i+2))(layer)

        layer = cell(units, activation=activation,
                    return_sequences=True, implementation=2, name='final_layer_of_rnn')(layer)
        layer = BatchNormalization(name='bt_rnn_final')(layer)
    

    time_dense = TimeDistributed(Dense(output_dim))(layer)
    # TODO: Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    # TODO: Specify model.output_length
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    print(model.summary())
    # plot_model(model, to_file='models/model_2.png', show_shapes=True)
    return model

In [32]:
def cnn_rnn_model(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=29):
    """ Build a recurrent + convolutional network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # Add convolutional layer
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='conv1d')(input_data)
    # Add batch normalization
    bn_cnn = BatchNormalization(name='bn_conv_1d')(conv_1d)
    # Add a recurrent layer
    simp_rnn = SimpleRNN(units, activation='relu',
        return_sequences=True,  name='rnn')(bn_cnn)
    # TODO: Add batch normalization
    
   
    bn_rnn = BatchNormalization()(simp_rnn)
    # TODO: Add a TimeDistributed(Dense(output_dim)) layer
    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    # model.output_length = lambda x: x
    print(model.summary())
    
    return model
    

In [36]:
def bidirectional_rnn_model_gpu(input_dim, units, output_dim=29):
    """ Build a bidirectional recurrent network for speech
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # TODO: Add bidirectional recurrent layer
    bidir_rnn = Bidirectional(GRU(units,
                                  return_sequences=True,
                                  implementation=2,
                                  name='bi_rnn',
                                  activation = 'tanh',
                                recurrent_activation = 'sigmoid',
                                recurrent_dropout = 0,
                                unroll = False,
                                use_bias = True,
                                 ))(input_data)

                                 
    bn_bidir_rnn = BatchNormalization( name='norm_bidir_rnn')(bidir_rnn)
    # TODO: Add a TimeDistributed(Dense(output_dim)) layer
    time_dense = TimeDistributed(Dense(output_dim))(bn_bidir_rnn)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [54]:
def bidirectional_rnn_model(input_dim, units, output_dim=29):
    """ Build a bidirectional recurrent network for speech
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # TODO: Add bidirectional recurrent layer
    bidir_rnn = Bidirectional(GRU(units,
                                  activation='softmax',
                                  return_sequences=True,
                                  implementation=2,
                                  name='bi_rnn'
                                 ))(input_data)
    bn_bidir_rnn = BatchNormalization( name='norm_bidir_rnn')(bidir_rnn)
    # TODO: Add a TimeDistributed(Dense(output_dim)) layer
    time_dense = TimeDistributed(Dense(output_dim))(bn_bidir_rnn)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [11]:
import mlflow
import mlflow.tensorflow

In [18]:
data=pd.read_json('../artifacts/meta.json')

In [19]:
data.text.count()

10179

In [21]:
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)

In [22]:
data.text.count()

9357

In [23]:
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [30]:
print(len(features),len(data.text.to_numpy()))

9357 9357


In [24]:
valid_data=pd.read_json("../artifacts/valid_meta.json")

In [25]:
valid_data.text.count()

1990

In [26]:
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)

In [27]:
valid_data.text.count()

1990

In [28]:
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

In [29]:
print(len(valid_features),len(valid_data.text.to_numpy()))

1990 1990


In [38]:
from Batch import Batch

In [49]:

MODEL_NAME = "bidirectional_rnn_gpu"
EPOCHS=10
MINI_BATCH_SIZE = 250
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

# Bidirectional RNN

In [47]:
bidirectional_rnn_gpu=bidirectional_rnn_model_gpu(input_dim=26, 
                        units=250)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 500)         417000    
_________________________________________________________________
norm_bidir_rnn (BatchNormali (None, None, 500)         2000      
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 29)          14529     
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0         
Total params: 433,529
Trainable params: 432,529
Non-trainable params: 1,000
_________________________________________________________________
None


In [40]:
from Train import train

In [50]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Bidirectional RNN GPU")
train(audio_gen, input_to_softmax=bidirectional_rnn_gpu, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

2021/08/09 14:12:43 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b627d8575e3440bb98bd1d94b37dda6d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
37/37 [==============================] - 44s 1s/step - loss: 160.6900 - val_loss: 249.7360
Epoch 2/10
37/37 [==============================] - 36s 971ms/step - loss: 155.5684 - val_loss: 204.4467
Epoch 3/10
37/37 [==============================] - 30s 808ms/step - loss: 153.1183 - val_loss: 197.6223
Epoch 4/10
37/37 [==============================] - 32s 876ms/step - loss: 151.1606 - val_loss: 166.5286
Epoch 5/10
37/37 [==============================] - 34s 922ms/step - loss: 149.4014 - val_loss: 176.8793
Epoch 6/10
37/37 [==============================] - 31s 836ms/step - loss: 148.5265 - val_loss: 157.5194
Epoch 7/10
37/37 [==============================] - 32s 882ms/step - loss: 146.9373 - val_loss: 167.6052
Epoch 8/10
37/37 [==============================] - 45s 1s/step - loss: 146.3483 - val_loss: 156.4237
Epoch 9/10
37/37 [==============================] - 30s 812ms/step - loss: 147.9628 - val_loss: 157.6867
Epoch 10/10
37/37 [==============================] - 30s 821m

2021/08/09 14:18:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


In [62]:

MODEL_NAME = "simple_rnn"
EPOCHS=10
MINI_BATCH_SIZE = 250
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

# Simple RNN

In [67]:
simple_rnn_model = simple_rnn(input_dim=26,
                units=5,
                activation='tanh',
                output_dim=29)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
rnn (GRU)                    (None, None, 5)           495       
_________________________________________________________________
batch_normalization_4 (Batch (None, None, 5)           20        
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 29)          174       
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0         
Total params: 689
Trainable params: 679
Non-trainable params: 10
_________________________________________________________________
None


In [68]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Simple RNN")
train(audio_gen, input_to_softmax=simple_rnn_model, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/09 14:35:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '822ca4f8a42f49e6a9915585f0ca1177', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
37/37 [==============================] - 28s 661ms/step - loss: 207.9806 - val_loss: 223.8220
Epoch 2/10
37/37 [==============================] - 25s 672ms/step - loss: 172.3360 - val_loss: 226.4976
Epoch 3/10
37/37 [==============================] - 34s 916ms/step - loss: 172.2853 - val_loss: 199.5817
Epoch 4/10
37/37 [==============================] - 25s 677ms/step - loss: 171.7939 - val_loss: 196.6637
Epoch 5/10
14/37 [==========>...................] - ETA: 11s - loss: 173.0780

# CNN RNN

In [33]:
cnn_rnn_model = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         26250     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 250)         1000      
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         90200     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 200)         800       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 29)          5829      
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0   